# Environment setup

In [ ]:
import pandas as pd
import pprint

# Testing questions set

In [ ]:
question = "Quando mi conviene gestire un articolo a PSO rispetto a pianificazione?"
question_out_of_scope = "Quando è morto Giulio Cesare?"
multiple_valid_questions = "Cosa significa che una fattura è in mancata consegna? Il cliente ha ricevuto la fattura?"
q_client = "Addebito bollo su nota credito. Su nota credito non mette più addebito bollo: precedente nota credito si."
q_client_without_object = "Su nota credito non mette più addebito bollo: precedente nota credito si."
q_rewritten = "Perché la nota di credito non sta aggiungendo più il bollo e come risolvere questo problema?"

In [ ]:
# Replace 'file_path.csv' with the path to your CSV file
file_path = 'real_evaluation_set.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path, sep = ';', encoding='utf-8')

In [ ]:
questions = [question, question_out_of_scope, multiple_valid_questions, q_client, q_client_without_object, q_rewritten]
questions_list = df['question'].tolist()
for q in questions_list:
    questions.append(q)
pprint.pprint(questions)
pprint.pprint(len(questions))

# Different promptings

## Zero shot

In [ ]:
template_zs_eng = """You are an AI language model acting as a customer support assistant for Panthera software.  
Your task is to generate five different versions of the Italian question provided by the user, in order to retrieve relevant documents from a vector database.  
The context involves technical manuals for business management software.  

**IMPORTANT:** Only process questions directly related to the context of business management software, technical manuals, or IT-related topics.  
If the question is unrelated to these contexts, **do not rewrite it**. In such cases, provide only the tag: "OUT-OF SCOPE".
If the original question contains multiple questions, rewrite only the ones relevant to the context and ignore entirely the unrelated ones, without giving the tag "OUT-OF SCOPE", which
is used only when no part of the original question is relevant to the software context.

**Instructions:**  
1. Check if the question is relevant to management software manuals or IT-related topics. If not, output only: "OUT-OF SCOPE".  
2. If the question is relevant, generate rewritten questions in Italian that maintain the original meaning, exploring different formulations and angles.  
3. Provide the rewritten questions in a bullet-point list separated by new lines.  
4. The output must contain only the rewritten questions, without explanations or comments.  

Perform the task only for questions relevant to the software context.  
Try to improve the original wording by exploring different angles that help clarify the problem or request, making the questions clearer and more readable for a general user.  

Original question: {question}  
"""

In [ ]:
template_zs_ita = """Sei un modello linguistico AI che svolge il ruolo di assistente clienti per il software Panthera. 
Il tuo compito è generare cinque versioni diverse della domanda fornita dall'utente per recuperare documenti rilevanti da un database vettoriale. 
Il contesto riguarda manuali tecnici di software per la gestione aziendale. Non fornire la riscrittura della domanda se non è relativa al contesto.
Se nessuna delle domande è rilevante per il contesto del software fornisci come output solamente il tag: "OUT-OF SCOPE".

**Istruzioni:**
1. Genera domande riscritte che mantengano il significato originale, esplorando diverse formulazioni e angolazioni. 
2. Ignora le domande che non sono pertinenti ai manuali di software gestionale o agli argomenti di informatica.
3. Fornisci le domande alternative in un elenco puntato separato da nuove righe.
4. L'output deve contenere solo le domande riscritte, senza spiegazioni o commenti.

Svolgi la task solo per le domande rilevanti al contesto del software. In questo caso, cerca di migliorare la formulazione originale 
esplorando diverse angolazioni che aiutino a comprendere meglio il problema o la richiesta, rendendo più chiare e leggibili le domande per un utente generico.

Domanda originale: {question} """

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_zs_eng = ChatPromptTemplate.from_template(template_zs_eng)
prompt_zs_ita = ChatPromptTemplate.from_template(template_zs_ita)

# Different models

## Llama3.2

In [ ]:
from langchain_ollama import ChatOllama

model_llama = ChatOllama(temperature= 0, model="llama3.2")

## Llama3.2 Instruct

In [ ]:
from langchain_ollama.llms import OllamaLLM

model_llama_instruct = OllamaLLM(model="llama3.2:3b-instruct-fp16", temperature=0)

## GPT-3.5-turbo

In [ ]:
import os
from langchain_openai import ChatOpenAI
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
model_gpt_old = ChatOpenAI(temperature=0, model="gpt-3.5-turbo") 

## GPT-4o

In [ ]:
from langchain_openai import ChatOpenAI
model_gpt = ChatOpenAI(temperature=0, model="gpt-4o") 

## Gemini

In [ ]:
import os
import google.generativeai as genai
from IPython.display import Markdown

genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [ ]:
%%capture
%pip install -qU langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0
)

# Question re-writing and evaluation

## Auxiliary functions

In [ ]:
from E2E_Evaluation_metrics import RAGEvaluator
from E2E_Evaluation_metrics import SemScoreQueryRewriting

evaluator = RAGEvaluator()
semscore = SemScoreQueryRewriting()
columns_to_drop = ['BLEU', 'ROUGE-2', 'ROUGE-L', 'BERT P', 'BERT R', 'Perplexity', 'Diversity']
metrics_columns = ['BERT F1', 'ROUGE-1', 'SemScore']
summary_csv_path = "model_performance_summary.csv"

In [ ]:
def generate_rewritten_questions(questions, model, language_prompting, prompting, data_frame, generator):
    """
    Generate rewritten questions for a list of questions and add them to a DataFrame.

    Parameters:
    - questions (list): List of original questions to rewrite.
    - model (str): Name of the model used for rewriting.
    - language_prompting (str): Language of prompting (e.g., "Ita").
    - prompting (str): Type of prompting (e.g., "Zero-shot").
    - data_frame (pd.DataFrame): The DataFrame to which new rows will be added.
    - generator (object): The generator object to invoke rewritten questions.

    Returns:
    - pd.DataFrame: Updated DataFrame with rewritten questions.
    """
    for question in questions:
        rewritten_questions = generator.invoke({"question": question})

        # If the question is out of scope, skip further processing
        if "OUT-OF SCOPE" in rewritten_questions:
            continue  # Skip out-of-scope questions entirely

        for alternative in rewritten_questions:
            # Create a new row with the necessary details
            new_row = pd.DataFrame([{
                "Original_Question": question,
                "Model": model,
                "Language-prompting": language_prompting, 
                "Prompting": prompting,
                "Rewritten_Question": alternative
            }])

            # Add the new row to the DataFrame
            data_frame = pd.concat([data_frame, new_row], ignore_index=True)
            data_frame = data_frame.drop_duplicates(ignore_index=True)

    return data_frame

In [ ]:
import pandas as pd

def evaluate_responses(eval_df, evaluator):
    results = []
    for idx, row in eval_df.iterrows():
        response = row['Rewritten_Question']
        reference = row['Original_Question']
        
        # Check if either response or reference is empty, and skip this row
        if not response or not reference:
            continue
        
        # Evaluate and store the results
        evaluation = evaluator.evaluate_all(response, reference)
        results.append(evaluation)
    
    # Convert results to a DataFrame
    eval_df = pd.DataFrame(results)
    return eval_df

In [ ]:
def process_evaluation_and_metrics(data_frame, evaluator, semscore, model_name, columns_to_drop):
    """
    Evaluate responses, compute semantic scores, and merge results into a DataFrame.

    Parameters:
    - data_frame (pd.DataFrame): The input DataFrame with original and rewritten questions.
    - evaluator (object): The evaluation object to compute BLEU, ROUGE, etc.
    - semscore (object): The semantic score computation object.
    - model_name (str): Name of the model for semantic similarity scoring.
    - columns_to_drop (list): List of columns to drop from the evaluated DataFrame.

    Returns:
    - pd.DataFrame: Updated DataFrame with evaluation metrics and semantic scores.
    """
    # Step 1: Evaluate responses
    eval_df = evaluate_responses(data_frame, evaluator)
    
    # Step 2: Drop unnecessary columns
    eval_df = eval_df.drop(columns=columns_to_drop, errors="ignore")

    # Step 3: Compute semantic scores
    cosine_similarities_bge, _ = semscore.compute_sem_score(data_frame, model_name=model_name)
    eval_df["SemScore"] = cosine_similarities_bge["Cosine_Similarity"]

    # Step 4: Merge original DataFrame with evaluation metrics
    merged_df = pd.concat([data_frame, eval_df], axis=1)

    return merged_df

In [ ]:
import pandas as pd

def aggregate_model_metrics(df, metrics_columns, group_column='Original_Question'):
    """
    This function aggregates model metrics by a specified group column, calculates the overall score,
    and computes the overall model performance across all groups (questions).

    Parameters:
    - df (pd.DataFrame): The DataFrame containing model metrics.
    - metrics_columns (list): A list of column names corresponding to the metrics to be aggregated.
    - group_column (str): The column name to group by (default is 'Original_Question').

    Returns:
    - aggregated_metrics (pd.DataFrame): A DataFrame with aggregated metrics for each group.
    - overall_model_performance (pd.Series): A Series with the overall model performance for each metric.
    """
    
    # Step 1: Group by the specified column and calculate the mean for each metric
    aggregated_metrics = df.groupby(group_column).agg({metric: 'mean' for metric in metrics_columns}).reset_index()

    # Step 2: Calculate an overall score for each group (optional)
    # You can customize how these metrics are combined; here, we'll take their average.
    aggregated_metrics['Overall_Score'] = aggregated_metrics[metrics_columns].mean(axis=1)

    # Step 3: Compute the overall model performance across all groups
    overall_model_performance = aggregated_metrics[metrics_columns].mean()

    return aggregated_metrics, overall_model_performance

In [ ]:
import pandas as pd
import os

def full_pipeline(
    questions, model, language_prompting, prompting, data_frame, generator,
    evaluator, semscore, model_name, columns_to_drop, metrics_columns,
    summary_csv_path
):
    """
    Full pipeline for generating rewritten questions, evaluating metrics, and aggregating results.

    Parameters:
    - questions (list): List of original questions.
    - model (str): Model name to annotate in the data.
    - language_prompting (str): Language for prompting annotation.
    - prompting (str): Prompting strategy annotation.
    - data_frame (pd.DataFrame): Initial DataFrame to append results.
    - generator (callable): Function or object to generate rewritten questions.
    - evaluator (object): Evaluation object for computing BLEU, ROUGE, etc.
    - semscore (object): Semantic score computation object.
    - model_name (str): Model name for semantic similarity scoring.
    - columns_to_drop (list): Columns to drop after evaluation.
    - metrics_columns (list): List of columns to use for aggregated metrics.
    - summary_csv_path (str): Path to the CSV file to store overall performance metrics.

    Returns:
    - tuple: (final DataFrame with all metrics, aggregated metrics DataFrame, overall model performance)
    """
    # Step 1: Generate rewritten questions
    updated_data_frame = generate_rewritten_questions(
        questions=questions,
        model=model,
        language_prompting=language_prompting,
        prompting=prompting,
        data_frame=data_frame,
        generator=generator
    )

    # Step 2: Evaluate metrics and add semantic similarity scores
    evaluated_data_frame = process_evaluation_and_metrics(
        data_frame=updated_data_frame,
        evaluator=evaluator,
        semscore=semscore,
        model_name=model_name,
        columns_to_drop=columns_to_drop
    )

    # Step 3: Aggregate metrics and compute overall performance
    aggregated_metrics, overall_model_performance = aggregate_model_metrics(
        evaluated_data_frame, metrics_columns
    )

    # Add model details to the overall performance
    overall_performance_row = overall_model_performance.to_frame().T
    overall_performance_row["Model"] = model
    overall_performance_row["Language_Prompting"] = language_prompting
    overall_performance_row["Prompting"] = prompting

    # Step 4: Save or update summary CSV
    if os.path.exists(summary_csv_path):
        # Load existing summary and append new results
        existing_summary = pd.read_csv(summary_csv_path)
        updated_summary = pd.concat([existing_summary, overall_performance_row], ignore_index=True)
        updated_summary = updated_summary.drop_duplicates(ignore_index=True)
    else:
        # Create new summary file
        updated_summary = overall_performance_row

    # Save updated summary to CSV
    updated_summary.to_csv(summary_csv_path, index=False)

    return evaluated_data_frame, aggregated_metrics, overall_model_performance, updated_summary

# Evaluate and compare models

In [ ]:
import pandas as pd

# Initialize a DataFrame
columns = [
    "Original_Question",    # The original question
    "Model",                # Model name
    "Language-prompting",   # Language
    "Prompting",            # Type of prompting
    "Rewritten_Question"   # Rewritten version of the question
]

# Create an empty DataFrame
data_llama_eng = pd.DataFrame(columns=columns)
data_llama_ita = pd.DataFrame(columns=columns)

data_llama_instruct_eng = pd.DataFrame(columns=columns)
data_llama_instruct_ita = pd.DataFrame(columns=columns)

data_gpt_old_eng = pd.DataFrame(columns=columns)
data_gpt_old_ita = pd.DataFrame(columns=columns)

data_gpt_eng = pd.DataFrame(columns=columns)
data_gpt_ita = pd.DataFrame(columns=columns)

data_gemini_eng = pd.DataFrame(columns=columns)
data_gemini_ita = pd.DataFrame(columns=columns)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

generate_queries_llama_eng = (prompt_zs_eng | model_llama | StrOutputParser() | (lambda x: x.split("\n")))
generate_queries_llama_ita = (prompt_zs_ita | model_llama | StrOutputParser() | (lambda x: x.split("\n")))

generate_queries_llama_instruct_eng = (prompt_zs_eng | model_llama_instruct | StrOutputParser() | (lambda x: x.split("\n")))
generate_queries_llama_instruct_ita = (prompt_zs_ita | model_llama_instruct | StrOutputParser() | (lambda x: x.split("\n")))

generate_queries_gpt_old_eng = (prompt_zs_eng | model_gpt_old | StrOutputParser() | (lambda x: x.split("\n")))
generate_queries_gpt_old_ita = (prompt_zs_ita | model_gpt_old | StrOutputParser() | (lambda x: x.split("\n")))

generate_queries_gpt_eng = (prompt_zs_eng | model_gpt | StrOutputParser() | (lambda x: x.split("\n")))
generate_queries_gpt_ita = (prompt_zs_ita | model_gpt | StrOutputParser() | (lambda x: x.split("\n")))

generate_queries_gemini_eng = (prompt_zs_eng | model_gemini | StrOutputParser() | (lambda x: x.split("\n")))
generate_queries_gemini_ita = (prompt_zs_ita | model_gemini | StrOutputParser() | (lambda x: x.split("\n")))

# Evaluate all models question rewriting capabilities

## Evaluate open source models

## Italian

In [ ]:
# Execute the full pipeline
df_llama_ita, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="llama3.2",
    language_prompting="Ita",
    prompting="Zero-shot",
    data_frame=data_llama_ita,
    generator=generate_queries_llama_ita,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_llama_ita.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
# Execute the full pipeline
df_llama_instruct_ita, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="llama3.2-instruct",
    language_prompting="Ita",
    prompting="Zero-shot",
    data_frame=data_llama_instruct_ita,
    generator=generate_queries_llama_instruct_ita,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_llama_instruct_ita.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

## English

In [ ]:
# Execute the full pipeline
df_llama_eng, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="llama3.2",
    language_prompting="Eng",
    prompting="Zero-shot",
    data_frame=data_llama_eng,
    generator=generate_queries_llama_eng,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_llama_eng.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
# Execute the full pipeline
df_llama_instruct_eng, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="llama3.2-instruct",
    language_prompting="Eng",
    prompting="Zero-shot",
    data_frame=data_llama_instruct_eng,
    generator=generate_queries_llama_instruct_eng,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_llama_instruct_eng.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

## Evaluate proprietary models

## Italian

In [ ]:
# Execute the full pipeline
df_gpt_old_ita, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gpt-3.5-turbo",
    language_prompting="Ita",
    prompting="Zero-shot",
    data_frame=data_gpt_old_ita,
    generator=generate_queries_gpt_old_ita,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gpt_old_ita.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
# Execute the full pipeline
df_gpt_ita, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gpt-4o",
    language_prompting="Ita",
    prompting="Zero-shot",
    data_frame=data_gpt_ita,
    generator=generate_queries_gpt_ita,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gpt_ita.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

## English

In [ ]:
# Execute the full pipeline
df_gpt_old_eng, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gpt-3.5-turbo",
    language_prompting="Eng",
    prompting="Zero-shot",
    data_frame=data_gpt_old_eng,
    generator=generate_queries_gpt_old_eng,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gpt_old_eng.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
# Execute the full pipeline
df_gpt_eng, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gpt-4o",
    language_prompting="Eng",
    prompting="Zero-shot",
    data_frame=data_gpt_eng,
    generator=generate_queries_gpt_eng,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gpt_eng.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
# Execute the full pipeline
df_gemini_ita, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gemini-1.5-pro",
    language_prompting="Ita",
    prompting="Zero-shot",
    data_frame=data_gemini_ita,
    generator=generate_queries_gemini_ita,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gemini_ita.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
# Execute the full pipeline
df_gemini_eng, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gemini-1.5-pro",
    language_prompting="Eng",
    prompting="Zero-shot",
    data_frame=data_gemini_eng,
    generator=generate_queries_gemini_eng,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=summary_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gemini_eng.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
import os
import re
plot_path = os.getcwd() + "/Plots"
plot_path

In [ ]:
# Function to save the plot with title-based filename and specified save path
def save_plot(fig, title, save_path=plot_path):
    """
    Save the plot to the specified directory with a filename based on the plot title.

    Args:
    fig (matplotlib.figure.Figure): The figure object of the plot.
    title (str): The title of the plot, used to generate the filename.
    save_path (str): Directory path where the plot should be saved (optional).
    """
    if save_path:
        # Sanitize the title to create a valid filename
        sanitized_title = re.sub(r"[^\w\s-]", "", title).replace(" ", "_")
        filename = f"{sanitized_title}.jpg"
        
        # Ensure the save_path exists, create if necessary
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        # Combine save_path with the filename
        full_save_path = os.path.join(save_path, filename)
        
        # Save the plot
        fig.savefig(full_save_path, format="jpg", dpi=300)
        print(f"Plot saved to {full_save_path}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

df = updated_summary.sort_values(by="Model")

# Combine Model and Language for x-tick labels
df["Model_Language"] = df["Model"] + " (" + df["Language_Prompting"] + ")"

# Set up the plot
x = np.arange(len(df["Model_Language"]))  # X-axis positions
width = 0.25  # Width of the bars

fig, ax = plt.subplots(figsize=(12, 6))

# Plot each metric
ax.bar(x - width, df["BERT F1"], width, label="BERT F1", color='blue')
ax.bar(x, df["ROUGE-1"], width, label="ROUGE-1", color='orange')
ax.bar(x + width, df["SemScore"], width, label="SemScore", color='green')

# Add labels and title
ax.set_xlabel("Model (Language)")
ax.set_ylabel("Score")
ax.set_title("Comparison of Metrics Across Models and Languages")
ax.set_xticks(x)
ax.set_xticklabels(df["Model_Language"], rotation=45, ha="right")
ax.legend()

# Show the plot
plt.tight_layout()

save_plot(fig, "Comparison of Metrics Across Models and Languages")

plt.show()

In [ ]:
# Function to create a barplot for a specific metric
def plot_metric(dataframe, metric, title, ylabel, color, save_path = plot_path):
    """
    Create a barplot for the specified metric.

    Args:
    dataframe (pd.DataFrame): The input DataFrame.
    metric (str): The column name for the metric to plot.
    title (str): The title of the plot.
    ylabel (str): The label for the y-axis.
    color (str): The color of the bars.
    """
    x = np.arange(len(dataframe["Model_Language"]))  # X-axis positions
    fig, ax = plt.subplots(figsize=(12, 8))
    bars=ax.bar(x, dataframe[metric], color=color, label=metric)

    # Add text annotations on bars
    for bar, value in zip(bars, dataframe[metric]):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
                f"{value:.3f}", ha="center", va="bottom", fontsize=10)
        
    ax.set_xlabel("Model (Language)")
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(dataframe["Model_Language"], rotation=45, ha="right")
    ax.legend()
    plt.tight_layout()

    # If a save_path is provided, combine it with the dynamically generated filename
    if save_path:
        # Sanitize the title to create a valid filename
        sanitized_title = re.sub(r"[^\w\s-]", "", title).replace(" ", "_")
        filename = f"{sanitized_title}.jpg"
        
        # Ensure the save_path exists, create if necessary
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        # Combine save_path with the filename
        full_save_path = os.path.join(save_path, filename)
        
        # Save the plot
        plt.savefig(full_save_path, format="jpg", dpi=300)
        print(f"Plot saved to {full_save_path}")

    plt.show()

# Create separate plots for each metric
plot_metric(df, "BERT F1", "General Comparison of BERT F1 Across Models", "BERT F1 Score", "blue")
plot_metric(df, "ROUGE-1", "General Comparison of ROUGE-1 Across Models", "ROUGE-1 Score", "orange")
plot_metric(df, "SemScore", "General Comparison of SemScore Across Models", "SemScore", "green")

In [ ]:
# Function to create a barplot for a specific metric and language
def plot_metric_by_language(dataframe, metric, title, ylabel, color, language, save_path = plot_path):
    """
    Create a barplot for the specified metric filtered by language.

    Args:
    dataframe (pd.DataFrame): The input DataFrame.
    metric (str): The column name for the metric to plot.
    title (str): The title of the plot.
    ylabel (str): The label for the y-axis.
    color (str): The color of the bars.
    language (str): The language to filter by (e.g., "Ita" or "Eng").
    """
    # Filter by language
    df_language = dataframe[dataframe["Language_Prompting"] == language]
    x = np.arange(len(df_language["Model_Language"]))  # X-axis positions
    fig, ax = plt.subplots(figsize=(12, 8))
    bars = ax.bar(x, df_language[metric], color=color, label=f"{metric} ({language})")

    # Add text annotations on bars
    for bar, value in zip(bars, df_language[metric]):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
                f"{value:.3f}", ha="center", va="bottom", fontsize=10)
        
    ax.set_xlabel("Model (Language)")
    ax.set_ylabel(ylabel)
    ax.set_title(f"{title} - {language.upper()} Models")
    ax.set_xticks(x)
    ax.set_xticklabels(df_language["Model_Language"], rotation=45, ha="right")
    ax.legend()
    plt.tight_layout()

    # If a save_path is provided, combine it with the dynamically generated filename
    if save_path:
        # Sanitize the title to create a valid filename
        sanitized_title = re.sub(r"[^\w\s-]", "", title).replace(" ", "_")
        filename = f"{sanitized_title}_{language}.jpg"
        
        # Ensure the save_path exists, create if necessary
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        # Combine save_path with the filename
        full_save_path = os.path.join(save_path, filename)
        
        # Save the plot
        plt.savefig(full_save_path, format="jpg", dpi=300)
        print(f"Plot saved to {full_save_path}")

    plt.show()

# Create separate plots for each metric and language
for language, color in [("Ita", "blue"), ("Eng", "orange")]:
    plot_metric_by_language(df, "BERT F1", "Comparison of BERT F1 Across Models", "BERT F1 Score", color, language)
    plot_metric_by_language(df, "ROUGE-1", "Comparison of ROUGE-1 Across Models", "ROUGE-1 Score", color, language)
    plot_metric_by_language(df, "SemScore", "Comparison of SemScore Across Models", "SemScore", color, language)

# Compare zero-shot, one-shot and few-shot prompting with gpt-4o

Let's analyze some possible questions, and manually create the rewritten versions, in order to use them as examples in the one-shot and few-shot prompting.

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the dataset
eval_dataset = Dataset.load_from_disk("eval_dataset")

# Convert to DataFrame
eval_df_syntethic = eval_dataset.to_pandas()
eval_df_syntethic = eval_df_syntethic[["question", "answer"]]
print(len(eval_df_syntethic))
display(eval_df_syntethic.head())

In [ ]:
import random
generated_questions = eval_df_syntethic['question']
sample = random.randint(0, len(generated_questions))
pprint.pprint(generated_questions[sample])

## One-shot prompting

In [ ]:
template_os_ita = """
Sei un modello linguistico AI che svolge il ruolo di assistente clienti per il software Panthera. 
Il tuo compito è generare cinque versioni diverse della domanda fornita dall'utente per recuperare documenti rilevanti da un database vettoriale. 
Il contesto riguarda manuali tecnici di software per la gestione aziendale. Non fornire la riscrittura della domanda se non è relativa al contesto.
Se nessuna delle domande è rilevante per il contesto del software gestionale fornisci come output solamente il tag: "OUT-OF SCOPE".

**Esempi:**
1. Domanda originale: "Come posso specificare il tipo costo da assegnare al nuovo ambiente di commessa?"
   Risposte:
   - Qual è il procedimento per definire il tipo di costo da applicare a un nuovo ambiente di commessa?
   - Come posso configurare il tipo di costo per un nuovo ambiente di commessa nel sistema?
   - Quali sono i passaggi per assegnare un tipo di costo a un ambiente di commessa appena creato?
   - Dove posso trovare l'opzione per impostare il tipo di costo di un nuovo ambiente di commessa?
   - Come posso scegliere il tipo di costo da associare a un ambiente di commessa nel mio sistema?

**Istruzioni:**
1. Genera domande riscritte che mantengano il significato originale, esplorando diverse formulazioni e angolazioni. 
2. Ignora le domande che non sono pertinenti ai manuali di software gestionale o agli argomenti di informatica.
3. Fornisci le domande alternative in un elenco puntato separato da nuove righe.
4. L'output deve contenere solo le domande riscritte, senza spiegazioni o commenti.

Svolgi la task solo per le domande rilevanti al contesto del software. In questo caso, cerca di migliorare la formulazione originale 
esplorando diverse angolazioni che aiutino a comprendere meglio il problema o la richiesta, rendendo più chiare e leggibili le domande per un utente generico. 

Domanda originale: {question}
"""

prompt_os_ita = ChatPromptTemplate.from_template(template_os_ita)

In [ ]:
data_gpt_os_ita = pd.DataFrame(columns = columns)
generate_queries_gpt_os_ita = (prompt_os_ita | model_gpt | StrOutputParser() | (lambda x: x.split("\n")))

In [ ]:
# Execute the full pipeline
prompting_type_csv_path = "prompting_type_csv_path_summary.csv"
df_gpt_os_ita, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gpt-4o",
    language_prompting="Ita",
    prompting="One-shot",
    data_frame=data_gpt_os_ita,
    generator=generate_queries_gpt_os_ita,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=prompting_type_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gpt_os_ita.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

## Few-shot prompting

In this case we won't generate only examples of acceptable questions and the rewritings, but we have also to include examples with not acceptable questions to which the model doesn't answer, but just retrurns the [OUT-OF SCOPE] tag; and we also consider multiple questions rewriting, with more valid questions, but also multiple questions with some valid ones and some not valid ones, even if we don't use these cases in the evaluation test-set, to not mislead the semantic scores due to some invalid question's parts. 

In [ ]:
import random
generated_questions = eval_df_syntethic['question']
sample = random.randint(0, len(generated_questions))
pprint.pprint(generated_questions[sample])

In [ ]:
template_fs_ita = """
Sei un modello linguistico AI che svolge il ruolo di assistente clienti per il software Panthera. 
Il tuo compito è generare cinque versioni diverse della domanda fornita dall'utente per recuperare documenti rilevanti da un database vettoriale. 
Il contesto riguarda manuali tecnici di software per la gestione aziendale. Non fornire la riscrittura della domanda se non è relativa al contesto.
Se nessuna delle domande è rilevante per il contesto del software fornisci come output solamente il tag: "OUT-OF SCOPE".

**Esempi:**
1. Domanda originale: "Come posso specificare il tipo costo da assegnare al nuovo ambiente di commessa?"
   Risposte:
   - Qual è il procedimento per definire il tipo di costo da applicare a un nuovo ambiente di commessa?
   - Come posso configurare il tipo di costo per un nuovo ambiente di commessa nel sistema?
   - Quali sono i passaggi per assegnare un tipo di costo a un ambiente di commessa appena creato?
   - Dove posso trovare l'opzione per impostare il tipo di costo di un nuovo ambiente di commessa?
   - Come posso scegliere il tipo di costo da associare a un ambiente di commessa nel mio sistema?

2. Domanda originale: "Come si fa la carbonara?"
   Risposta: OUT-OF SCOPE

3. Domanda originale: "Come posso definire lo stato di un articolo in questo software?"
   Risposte:
   - Qual è il procedimento per impostare lo stato di un articolo in questo software?
   - Come posso configurare lo stato di un articolo all'interno di questo software?
   - Dove posso definire o modificare lo stato di un articolo nel software?
   - Quali sono le opzioni per determinare lo stato di un articolo in questo software?
   - Come posso aggiornare lo stato di un articolo nel sistema?

4. Domanda originale: "Ho un errore, come posso risolvere il problema delle schermate nere all'avvio del software? E come posso fare la carbonara?"
   Risposte: 
   - Come posso risolvere il problema delle schermate nere che si verificano all'avvio del software?
   - Cosa devo fare per correggere l'errore delle schermate nere durante l'avvio del programma?
   - Quali sono le possibili cause delle schermate nere all'avvio del software e come posso risolvere il problema?
   - Come posso risolvere il malfunzionamento delle schermate nere che si presentano quando avvio il software?
   - Perché vedo delle schermate nere all'avvio del software e quali sono le soluzioni disponibili?

**Istruzioni:**
1. Genera domande riscritte che mantengano il significato originale, esplorando diverse formulazioni e angolazioni. 
2. Ignora le domande che non sono pertinenti ai manuali di software gestionale o agli argomenti di informatica.
3. Fornisci le domande alternative in un elenco puntato separato da nuove righe.
4. L'output deve contenere solo le domande riscritte, senza spiegazioni o commenti.

Svolgi la task solo per le domande rilevanti al contesto del software. In questo caso, cerca di migliorare la formulazione originale 
esplorando diverse angolazioni che aiutino a comprendere meglio il problema o la richiesta, rendendo più chiare e leggibili le domande per un utente generico. 

Domanda originale: {question}
"""

prompt_fs_ita = ChatPromptTemplate.from_template(template_fs_ita)
data_gpt_fs_ita = pd.DataFrame(columns = columns)
generate_queries_gpt_fs_ita = (prompt_fs_ita | model_gpt | StrOutputParser() | (lambda x: x.split("\n")))

In [ ]:
# Execute the full pipeline
df_gpt_fs_ita, aggregated_metrics, overall_model_performance, updated_summary = full_pipeline(
    questions=questions,
    model="gpt-4o",
    language_prompting="Ita",
    prompting="Few-shot",
    data_frame=data_gpt_fs_ita,
    generator=generate_queries_gpt_fs_ita,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=prompting_type_csv_path
)

# Display the results
print("Single Metrics for Each Question and Rewritten Question:")
display(df_gpt_fs_ita.head())

print("Aggregated Metrics for Each Question:")
display(aggregated_metrics)

print("\nOverall Model Performance:")
display(overall_model_performance)

print("Updated Model Performance Summary:")
display(updated_summary)

In [ ]:
len(df_gpt_ita), len(df_gpt_os_ita), len(df_gpt_fs_ita)

In [ ]:
# Identify rows to drop based on the condition
rows_to_drop = df_gpt_ita[~df_gpt_ita['Original_Question'].isin(df_gpt_fs_ita['Original_Question'].unique())].index

# Drop the rows from df_gpt_ita
df_gpt_ita_new = df_gpt_ita.drop(rows_to_drop).reset_index(drop=True)

In [ ]:
 # Step 3: Aggregate metrics and compute overall performance
aggregated_metrics, overall_model_performance = aggregate_model_metrics(
    df_gpt_ita_new, metrics_columns
)

 # Add model details to the overall performance
overall_performance_row = overall_model_performance.to_frame().T
overall_performance_row["Model"] = "gpt-4o"
overall_performance_row["Language_Prompting"] = "Ita"
overall_performance_row["Prompting"] = "Zero-shot"

updated_summary = pd.concat([updated_summary, overall_performance_row], ignore_index=True)
updated_summary = updated_summary.drop_duplicates(ignore_index=True)
updated_summary

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is already defined and contains the data
df = updated_summary

# Filter data for gpt-4o models and Language_Prompting 'Ita'
filtered_df = df[(df["Model"] == "gpt-4o") & (df["Language_Prompting"] == "Ita")]

# Melt the DataFrame for easier plotting
melted_df = filtered_df.melt(id_vars=["Prompting"], value_vars=["BERT F1", "ROUGE-1", "SemScore"], 
                             var_name="Metric", value_name="Score")

# Define the desired order of prompting types
prompting_order = ["Zero-shot", "One-shot", "Few-shot"]

# Plot using seaborn with the specified order
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=melted_df, x="Prompting", y="Score", hue="Metric", palette="viridis", order=prompting_order)

# Add text annotations for each bar
for container in ax.containers:
    for bar in container:
        bar_height = bar.get_height()
        if bar_height > 0:  # Only annotate if the bar's height is greater than 0
            ax.text(
                bar.get_x() + bar.get_width() / 2,  # X-coordinate (bar center)
                bar_height,  # Y-coordinate (just above the bar)
                f'{bar_height:.3f}',  # Text to display (score with 3 decimals)
                ha='center', va='bottom', fontsize=9, color='black'
            )

# Customize plot
plot_filename = "Scores_for_gpt-4o_Models_with_Ita_Language_Prompting.jpg"  # Name of the file
plt.title("Scores for gpt-4o Models with Ita Language_Prompting", fontsize=14)
plt.xlabel("Prompting Type", fontsize=12)
plt.ylabel("Score", fontsize=12)
plt.legend(title="Metrics", fontsize=10)
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Save the plot to a file
output_path = f"{plot_path}/{plot_filename}"
plt.tight_layout()
plt.savefig(output_path, dpi=300)
plt.show()

print(f"Plot saved successfully to {output_path}")

# Let's analyze also the number of useful few-shot elements

In [ ]:
template_i = """
Sei un modello linguistico AI che svolge il ruolo di assistente clienti per il software Panthera. 
Il tuo compito è generare cinque versioni diverse della domanda fornita dall'utente per recuperare documenti rilevanti da un database vettoriale. 
Il contesto riguarda manuali tecnici di software per la gestione aziendale. Non fornire la riscrittura della domanda se non è relativa al contesto.
Se nessuna delle domande è rilevante per il contesto del software fornisci come output solamente il tag: "OUT-OF SCOPE".

**Esempi:**
1. Domanda originale: "Come posso specificare il tipo costo da assegnare al nuovo ambiente di commessa?"
   Risposte:
   - Qual è il procedimento per definire il tipo di costo da applicare a un nuovo ambiente di commessa?
   - Come posso configurare il tipo di costo per un nuovo ambiente di commessa nel sistema?
   - Quali sono i passaggi per assegnare un tipo di costo a un ambiente di commessa appena creato?
   - Dove posso trovare l'opzione per impostare il tipo di costo di un nuovo ambiente di commessa?
   - Come posso scegliere il tipo di costo da associare a un ambiente di commessa nel mio sistema?

2. Domanda originale: "Come si fa la carbonara?"
   Risposta: OUT-OF SCOPE

3. Domanda originale: "Come posso definire lo stato di un articolo in questo software?"
   Risposte:
   - Qual è il procedimento per impostare lo stato di un articolo in questo software?
   - Come posso configurare lo stato di un articolo all'interno di questo software?
   - Dove posso definire o modificare lo stato di un articolo nel software?
   - Quali sono le opzioni per determinare lo stato di un articolo in questo software?
   - Come posso aggiornare lo stato di un articolo nel sistema?

4. Domanda originale: "Ho un errore, come posso risolvere il problema delle schermate nere all'avvio del software? E come posso fare la carbonara?"
   Risposte: 
   - Come posso risolvere il problema delle schermate nere che si verificano all'avvio del software?
   - Cosa devo fare per correggere l'errore delle schermate nere durante l'avvio del programma?
   - Quali sono le possibili cause delle schermate nere all'avvio del software e come posso risolvere il problema?
   - Come posso risolvere il malfunzionamento delle schermate nere che si presentano quando avvio il software?
   - Perché vedo delle schermate nere all'avvio del software e quali sono le soluzioni disponibili?

5. Domanda originale: "Quali informazioni posso visualizzare riguardo ai beni ammortizzabili?"
   Risposte:
   - Quali dettagli sono disponibili nel sistema sui beni ammortizzabili?
   - Che tipo di informazioni posso consultare sui beni ammortizzabili nel software?
   - Quali dati relativi ai beni ammortizzabili possono essere visualizzati o gestiti?
   - Dove posso trovare le informazioni disponibili sui beni ammortizzabili?
   - Quali informazioni sui beni ammortizzabili sono accessibili attraverso il sistema?

6. Domanda originale: "Come posso gestire le quantità in unità di misura intere nel magazzino? Dove si trova fisicamente il magazzino?"
   Risposte:
   - Come posso gestire le quantità in unità di misura intere nel sistema di magazzino?
   - Qual è il metodo per gestire le quantità in unità intere nel magazzino?
   - Come posso configurare il magazzino per gestire le quantità in unità di misura intere?
   - Quali sono le opzioni per gestire le quantità in unità intere nel magazzino?
   - Come posso impostare la gestione delle quantità in unità intere nel magazzino per il mio sistema?

7. Domanda originale: "Buonasera, qual è il fattore di scarto considerato dal sistema durante la fase di estazione pso? Si può cambiare in qualche modo?"
   Risposte:
   - Qual è il fattore di scarto utilizzato dal sistema durante la fase di estazione PSO e come posso modificarlo?
   - Come viene calcolato il fattore di scarto nella fase di estazione PSO e c'è la possibilità di cambiarlo?
   - In che modo il sistema considera il fattore di scarto durante la fase di estazione PSO e può essere personalizzato?
   - Qual è il valore predefinito del fattore di scarto nella fase di estazione PSO e come posso cambiarlo?
   - È possibile modificare il fattore di scarto usato dal sistema nella fase di estazione PSO? Se sì, come?

8. Domanda originale: "Voglio copiare le configurazioni di un articolo in un altro con lo stesso schema di configurazione?"
   Risposte:
   - Come posso copiare le configurazioni di un articolo e applicarle a un altro con lo stesso schema di configurazione?
   - Esiste un metodo per trasferire le configurazioni di un articolo su un altro mantenendo lo stesso schema?
   - Qual è la procedura per duplicare le configurazioni di un articolo e usarle per un altro con lo stesso schema?
   - Come posso replicare le impostazioni di un articolo in un altro, mantenendo invariato lo schema di configurazione?
   - È possibile copiare e incollare le configurazioni di un articolo in un altro con lo stesso schema di configurazione?

9. Domanda originale: "Mi puoi aiutare ad integrare una fattura ricevuta con la relativa aliquota e imposta?"
   Risposte:
   - Come posso integrare una fattura ricevuta nel sistema includendo l'aliquota e l'imposta applicata?
   - Qual è la procedura per registrare una fattura ricevuta e calcolare correttamente l'aliquota e l'imposta?
   - Come posso associare l'aliquota e l'imposta a una fattura ricevuta nel mio sistema?
   - C'è un modo per aggiungere l'aliquota e l'imposta a una fattura ricevuta in modo automatico nel software?
   - In che modo posso integrare una fattura con l'aliquota e l'imposta per un calcolo corretto?

**Istruzioni:**
1. Genera domande riscritte che mantengano il significato originale, esplorando diverse formulazioni e angolazioni. 
2. Ignora le domande che non sono pertinenti ai manuali di software gestionale o agli argomenti di informatica.
3. Fornisci le domande alternative in un elenco puntato separato da nuove righe.
4. L'output deve contenere solo le domande riscritte, senza spiegazioni o commenti.

Svolgi la task solo per le domande rilevanti al contesto del software. In questo caso, cerca di migliorare la formulazione originale 
esplorando diverse angolazioni che aiutino a comprendere meglio il problema o la richiesta, rendendo più chiare e leggibili le domande per un utente generico. 

Domanda originale: {question}
"""

prompt_i = ChatPromptTemplate.from_template(template_i)
data_gpt_prompt_i = pd.DataFrame(columns = columns)
generate_queries_prompt_i = (prompt_i | model_gpt | StrOutputParser() | (lambda x: x.split("\n")))

# Execute the full pipeline
prompting_i_examples = "prompting_i_examples.csv"
df_prompt_i, aggregated_metrics, overall_model_performance, examples_i_summary = full_pipeline(
    questions=questions,
    model="gpt-4o",
    language_prompting="Ita",
    prompting="Few-shot-9",
    data_frame=data_gpt_prompt_i,
    generator=generate_queries_prompt_i,
    evaluator=evaluator,
    semscore=semscore,
    model_name='BAAI/bge-m3',
    columns_to_drop=columns_to_drop,
    metrics_columns=metrics_columns, 
    summary_csv_path=prompting_i_examples
)

print("Updated Model Performance Summary:")
display(examples_i_summary)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming examples_i_summary is a predefined data structure
df = pd.DataFrame(examples_i_summary)

# Create subplots for each metric
fig, axs = plt.subplots(3, 1, figsize=(10, 18))  # 3 rows, 1 column

# Plotting BERT F1
axs[0].plot(df["Prompting"], df["BERT F1"], label="BERT F1", marker='o', color='blue')
axs[0].set_title("BERT F1 Performance")
axs[0].set_xlabel("Number of Few-shot Examples")
axs[0].set_ylabel("BERT F1 Score")
axs[0].legend()
axs[0].grid(True)
axs[0].set_ylim([df["BERT F1"].min() - 0.1, df["BERT F1"].max() + 0.1])  # Zoom on BERT F1 values

# Annotate all BERT F1 points
for i, val in enumerate(df["BERT F1"]):
    if i == df["BERT F1"].idxmax():
        axs[0].annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, -15), ha='center', color='blue')
    else:
        axs[0].annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, 10), ha='center', color='black')

# Plotting ROUGE-1
axs[1].plot(df["Prompting"], df["ROUGE-1"], label="ROUGE-1", marker='o', color='green')
axs[1].set_title("ROUGE-1 Performance")
axs[1].set_xlabel("Number of Few-shot Examples")
axs[1].set_ylabel("ROUGE-1 Score")
axs[1].legend()
axs[1].grid(True)
axs[1].set_ylim([df["ROUGE-1"].min() - 0.1, df["ROUGE-1"].max() + 0.1])  # Zoom on ROUGE-1 values

# Annotate all ROUGE-1 points
for i, val in enumerate(df["ROUGE-1"]):
    if i == df["ROUGE-1"].idxmax():
        axs[1].annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, -15), ha='center', color='green')
    else:
        axs[1].annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, 10), ha='center', color='black')

# Plotting Semantic Score
axs[2].plot(df["Prompting"], df["SemScore"], label="Semantic Score", marker='o', color='red')
axs[2].set_title("Semantic Score Performance")
axs[2].set_xlabel("Number of Few-shot Examples")
axs[2].set_ylabel("Semantic Score")
axs[2].legend()
axs[2].grid(True)
axs[2].set_ylim([df["SemScore"].min() - 0.1, df["SemScore"].max() + 0.1])  # Zoom on Semantic Score values

# Annotate all Semantic Score points
for i, val in enumerate(df["SemScore"]):
    if i == df["SemScore"].idxmax():
        axs[2].annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, -15), ha='center', color='red')
    else:
        axs[2].annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, 10), ha='center', color='black')

# Save the Semantic Score plot
axs[2].figure.savefig(f"{plot_path}/Metrics_Performance_Few_shot.jpg")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming examples_i_summary is a predefined data structure
df = pd.DataFrame(examples_i_summary)

# Calculate the average of the three metrics
df["Average"] = df[["BERT F1", "ROUGE-1", "SemScore"]].mean(axis=1)

# Find the index of the maximum average value
max_avg_idx = df["Average"].idxmax()

# Plotting the scores for analysis
plt.figure(figsize=(10, 6))

# Plot the individual metrics
plt.plot(df["Prompting"], df["BERT F1"], label="BERT F1", marker='o', color='blue')
plt.plot(df["Prompting"], df["ROUGE-1"], label="ROUGE-1", marker='o', color='green')
plt.plot(df["Prompting"], df["SemScore"], label="Semantic Score", marker='o', color='red')

# Plot the average values
plt.plot(df["Prompting"], df["Average"], label="Average", marker='o', color='orange', linestyle='--')

# Annotate the average values
for i, val in enumerate(df["Average"]):
    if i == max_avg_idx:
        # Annotate max average in a distinct color
        plt.annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, 10), 
                     ha='center', color='orange', fontsize=10, weight='bold')
    else:
        # Annotate other averages in black
        plt.annotate(f"{val:.4f}", (df["Prompting"][i], val), textcoords="offset points", xytext=(10, -10), 
                     ha='center', color='black', fontsize=8)

# Add a vertical line for the `Prompting` value with max average
plt.axvline(x=df["Prompting"][max_avg_idx], color='grey', linestyle='--', label='Max Avg Prompting')

# Set plot titles and labels
plt.title("Performance of GPT-4o Model with Few-shot Examples")
plt.xlabel("Number of Few-shot Examples")
plt.ylabel("Score")
plt.legend()
plt.grid()

# Save the plot with the title as the filename
plt.savefig(f"{plot_path}/Compare_Performance_GPT-4o_Different_Few-Shot_Examples.jpg")

# Show the plot
plt.show()